In [29]:
def print_structure(var, indent=0):
    """
    Recursively prints the structure of the given Python variable.
    """
    prefix = " " * indent  # Indentation
    if isinstance(var, dict):
        print(f"{prefix}{type(var)} containing:")
        for key, value in var.items():
            print(f"{prefix}  Key: {key} ->", end=" ")
            print_structure(value, indent + 4)
    elif isinstance(var, list):
        print(f"{prefix}{type(var)} with {len(var)} elements:")
        for i, item in enumerate(var):
            print(f"{prefix}  Index {i} ->", end=" ")
            print_structure(item, indent + 4)
    elif isinstance(var, tuple):
        print(f"{prefix}{type(var)} with {len(var)} elements:")
        for i, item in enumerate(var):
            print(f"{prefix}  Element {i} ->", end=" ")
            print_structure(item, indent + 4)
    elif isinstance(var, set):
        print(f"{prefix}{type(var)} with {len(var)} elements:")
        for item in var:
            print(f"{prefix}  Value ->", end=" ")
            print_structure(item, indent + 4)
    else:
        print(f"{prefix}{type(var)} with value {var}")

# Example usage
example_variable = {
    "polygons": [
        {
            "label": "a",
            "points": [{"x": 100, "y": 10}, {"x": 60, "y": 60}],
        },
        {
            "label": "b",
            "points": [{"x": 200, "y": 20}, {"x": 160, "y": 160}],
        },
    ],
    "settings": {
        "threshold": 0.8,
        "method": "TM_CCOEFF"
    },
    "misc": ["item1", "item2"],
    "tuple_example": (1, 2, 3),
    "set_example": {1, 2, 3}
}

print_structure(example_variable)


<class 'dict'> containing:
  Key: polygons ->     <class 'list'> with 2 elements:
      Index 0 ->         <class 'dict'> containing:
          Key: label ->             <class 'str'> with value a
          Key: points ->             <class 'list'> with 2 elements:
              Index 0 ->                 <class 'dict'> containing:
                  Key: x ->                     <class 'int'> with value 100
                  Key: y ->                     <class 'int'> with value 10
              Index 1 ->                 <class 'dict'> containing:
                  Key: x ->                     <class 'int'> with value 60
                  Key: y ->                     <class 'int'> with value 60
      Index 1 ->         <class 'dict'> containing:
          Key: label ->             <class 'str'> with value b
          Key: points ->             <class 'list'> with 2 elements:
              Index 0 ->                 <class 'dict'> containing:
                  Key: x ->              

In [33]:
import cv2
import numpy as np
from PIL import Image

def find_matches(img, template, polygons, min_distance):
    # Convert PIL images to NumPy arrays for OpenCV processing
    img_np = np.array(img)
    template_np = np.array(template)

    # Convert to grayscale
    img_gray = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
    template_gray = cv2.cvtColor(template_np, cv2.COLOR_BGR2GRAY)
    
    # Perform template matching
    res = cv2.matchTemplate(img_gray, template_gray, cv2.TM_CCOEFF_NORMED)
    
    # Set a threshold for detecting matches
    threshold = 0.6
    loc = np.where(res >= threshold)
    
    new_polygons = []
    for pt in zip(*loc[::-1]):  # loc gives us the top-left corner of the match
        match_center = (pt[0] + template_np.shape[1]//2, pt[1] + template_np.shape[0]//2)  # Calculate match center
        if is_far_enough(match_center, polygons, min_distance):
            # If far enough, add to new polygons list
            new_polygons.append({
                "points": [
                    {"x": pt[0], "y": pt[1]},
                    {"x": pt[0] + template_np.shape[1], "y": pt[1]},
                    {"x": pt[0] + template_np.shape[1], "y": pt[1] + template_np.shape[0]},
                    {"x": pt[0], "y": pt[1] + template_np.shape[0]}
                ],
                "label": "detected"
            })
    
    return new_polygons

def is_far_enough(point, polygons, min_distance):
    for polygon in polygons:
        for p in polygon["points"]:
            # Calculate Euclidean distance between two points
            distance = np.sqrt((p["x"] - point[0])**2 + (p["y"] - point[1])**2)
            if distance < min_distance:
                return False
    return True


In [34]:
from PIL import Image, ImageDraw
import os
import json

with open('polygons.json', 'r') as f:
    polygons = json.load(f)
    #print(polygons)
    print_structure(polygons)
    
    # Convert polygon points to a format suitable for PIL (list of tuples)
    for index, polygon in enumerate(polygons):
        if polygon['label'] != "a":
            continue
            
        polygon_points = [(int(point["x"]), int(point["y"])) for point in polygon['points']]
        
        # Determine the bounding box of the polygon
        min_x = min(point[0] for point in polygon_points)
        min_y = min(point[1] for point in polygon_points)
        max_x = max(point[0] for point in polygon_points)
        max_y = max(point[1] for point in polygon_points)
        
        # Create a new image with white background
        img_size = (max_x - min_x, max_y - min_y)
        img = Image.new("RGB", img_size, "white")
        
        # Open the source image
        source_img_path = os.path.join("static", "1.jpg")
        source_img = Image.open(source_img_path)
        
        # Calculate the offset to crop the source image correctly
        offset = (min_x, min_y)
        
        # Crop the source image to the bounding box of the polygon
        source_img_cropped = source_img.crop((min_x, min_y, max_x, max_y))
        
        # Paste the cropped source image onto the new image
        img.paste(source_img_cropped, (0, 0))
        
        # Draw the polygon on the new image
        draw = ImageDraw.Draw(img)
        draw.polygon(polygon_points, outline="red")


        # Load the image from the path
        image_path = "static/1.jpg"
        reference = Image.open(image_path)
        matches = find_matches(reference, img, polygons, 10)
        print(matches)
        
        # Save the result
        img.save("result"+str(index)+".png")     
    

<class 'list'> with 5 elements:
  Index 0 ->     <class 'dict'> containing:
      Key: label ->         <class 'str'> with value a
      Key: points ->         <class 'list'> with 4 elements:
          Index 0 ->             <class 'dict'> containing:
              Key: x ->                 <class 'float'> with value 686.8764956975363
              Key: y ->                 <class 'float'> with value 623.7642910911253
          Index 1 ->             <class 'dict'> containing:
              Key: x ->                 <class 'float'> with value 673.9652777989596
              Key: y ->                 <class 'float'> with value 656.042335837567
          Index 2 ->             <class 'dict'> containing:
              Key: x ->                 <class 'float'> with value 689.3594222164933
              Key: y ->                 <class 'float'> with value 655.5457505337755
          Index 3 ->             <class 'dict'> containing:
              Key: x ->                 <class 'float'> wit